In [11]:
import boto3
import pandas as pd

In [11]:
session = boto3.Session(profile_name='default') # requires a local ~/.aws/credentials file w/ permission
s3_client = session.client('s3')

In [59]:
# download some metadata we want to plug in
remotePath = 'dtingley/processed/metaMatrix.csv'
file = 'metaMatrix.csv'
s3_client.download_file(bucket, remotePath, file)
metaMatrix = pd.read_table('metaMatrix.csv',sep=',',index_col=0)
metaMatrix.head()

,Loyal_Id,Age_Yrs,Age_Mths,Breed_Avg_Lifespan,Energetic_Enthusiastic,Happy_Content,Active_Comfortable,Calm_Relaxed,Canine_Frailty_Score,Canine_Frailty_Total,...,ltt_submitted,tracking_number,entered_weight,weight_lbs,weight_kgs,entered_temperature,temperature_fahrenheit,temperature_celsius,heart_rate,body_condition_score
BAK-27,56470.0,5.7,68.0,11.0,3.23,6.00,5.62,2.19,0.02,0.5,...,1.0,8.164446e+11,97.8,97.8,44.4,101.8,101.8,38.8,100.0,7.0
BAK-38,53815.0,8.1,97.0,11.0,NaN,NaN,NaN,NaN,0.12,4.0,...,1.0,8.164446e+11,109.0,109.0,49.4,103.1,103.1,39.5,100.0,6.0
BAK-39,53624.0,13.6,163.0,NaN,1.15,3.98,2.76,1.96,0.26,8.5,...,1.0,8.165731e+11,80.0,80.0,36.3,101.9,101.9,38.8,140.0,6.0
BAK-40,54002.0,7.4,89.0,11.0,3.45,5.64,5.67,2.65,0.05,1.5,...,1.0,8.165731e+11,66.8,66.8,30.3,101.3,101.3,38.5,88.0,6.0
BAK-51,56603.0,4.8,58.0,11.0,2.84,6.00,5.11,2.62,0.08,2.5,...,1.0,8.165731e+11,68.6,68.6,31.1,103.0,103.0,39.4,120.0,6.0


In [84]:
bucket = 'bioinformatics-loyal'
remotePath ='processed_methylation_data/combined_datasets/all_saliva/matrices_processed/methylation_filtered.gctx_n98x997979.gctx'
file = 'methylation_filtered.gctx_n98x997979.gctx'

s3_client.download_file(bucket, remotePath, file)

In [87]:
from cmapPy.pandasGEXpress.parse import parse

my_col_metadata = parse(file, col_meta_only=True)
my_row_metadata = parse(file, row_meta_only=True)

In [88]:
# CpG site information
my_row_metadata

rhd,chrom,end,start
rid,,,
chr1_4899_4900,chr1,4900,4899
chr1_4913_4914,chr1,4914,4913
chr1_4926_4927,chr1,4927,4926
chr1_4932_4933,chr1,4933,4932
chr1_5803_5804,chr1,5804,5803
...,...,...,...
chrX_122709830_122709831,chrX,122709831,122709830
chrX_122733040_122733041,chrX,122733041,122733040
chrX_122733085_122733086,chrX,122733086,122733085


In [89]:
# sample metadata, this seems like the place to add metadata fields..
my_col_metadata

chd,file,sample_id,sequence_id
cid,,,
LOF-65_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,LOF-65_saliva,21152XD-01-01_S1_L004
CLA-36_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,CLA-36_saliva,21152XD-01-02_S0_L001
LOF-17_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,LOF-17_saliva,21152XD-01-03_S3_L004
LOF-64_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,LOF-64_saliva,21152XD-01-04_S0_L001
SHU-38_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,SHU-38_saliva,21152XD-01-05_S5_L004
...,...,...,...
HAG-11_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,HAG-11_saliva,bryn_palmer
HAG-08_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,HAG-08_saliva,powder_gaur
HAG-15_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,HAG-15_saliva,puck


In [66]:
metaMatrix['sample_id'] = metaMatrix.index + '_saliva' 

In [67]:
my_col_metadata = pd.merge(my_col_metadata,metaMatrix,how='outer',on='sample_id')

In [69]:
my_col_metadata['cid'] = my_col_metadata['sample_id']
my_col_metadata = my_col_metadata.set_index('cid')
my_col_metadata.head()

,file,sample_id,sequence_id,Loyal_Id,Age_Yrs,Age_Mths,Breed_Avg_Lifespan,Energetic_Enthusiastic,Happy_Content,Active_Comfortable,...,ltt_submitted,tracking_number,entered_weight,weight_lbs,weight_kgs,entered_temperature,temperature_fahrenheit,temperature_celsius,heart_rate,body_condition_score
cid,,,,,,,,,,,,,,,,,,,,,
LOF-65_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,LOF-65_saliva,21152XD-01-01_S1_L004,65410.0,10.1,121.0,11.0,3.12,6.00,5.80,...,1.0,8.165768e+11,37.3,82.2,37.3,101.3,101.3,38.5,80.0,5.0
CLA-36_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,CLA-36_saliva,21152XD-01-02_S0_L001,65177.0,8.6,103.0,NaN,3.21,5.54,4.95,...,1.0,8.164446e+11,59.0,59.0,26.8,100.4,100.4,38.0,120.0,5.0
LOF-17_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,LOF-17_saliva,21152XD-01-03_S3_L004,67036.0,2.0,24.0,11.0,4.15,6.00,6.00,...,1.0,8.165768e+11,32.0,70.5,32.0,101.0,101.0,38.3,80.0,5.0
LOF-64_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,LOF-64_saliva,21152XD-01-04_S0_L001,65411.0,14.9,179.0,11.0,1.45,4.37,4.06,...,1.0,8.165768e+11,29.8,65.7,29.8,100.8,100.8,38.2,80.0,6.0
SHU-38_saliva,s3://bioinformatics-loyal/nf-core_processing/H...,SHU-38_saliva,21152XD-01-05_S5_L004,57561.0,3.8,45.0,NaN,2.97,5.90,5.80,...,1.0,8.165731e+11,65.6,65.6,29.8,101.0,101.0,38.3,70.0,5.0


In [71]:
# TODO, write back to GCTX file one we figure out the data matrix bug w/ parse.py????


In [ ]:
gctobj = pdg.GCToo.GCToo(data_df, row_metadata_df=my_row_metadata, col_metadata_df=my_col_metadata)

pdg.write_gctx.write(gctobj,'test_output.gctx')